In [30]:
import copy
import re
from pathlib import Path
import os
import hjson
import numpy as np
import pandas as pd
from IPython.display import display
from loguru import logger
import time
import re

# Visualization
from phd_visualizations.test_timeseries import experimental_results_plot
from phd_visualizations import save_figure
from phd_visualizations.constants import generate_plotly_config

# Setup environment for running MATLAB code if not done externally
# os.environ["MR"] = f"{os.environ['HOME']}/PSA/MATLAB_runtime/R2023b"
os.environ["MR"] = f"{os.environ['HOME']}/MATLAB/R2023b"
MR = os.environ["MR"]
os.environ["LD_LIBRARY_PATH"] = f"{MR}/runtime/glnxa64:{MR}/bin/glnxa64:{MR}/sys/os/glnxa64:{MR}/sys/opengl/lib/glnxa64"

# auto reload modules
%load_ext autoreload

# Paths definition
src_path = Path(f'{os.getenv("HOME")}/Nextcloud/Juanmi_MED_PSA/EURECAT/')

# Resample figures using plotly_resampler
resample_figures = False

# sample_rate = '60s'
# sample_rate_numeric = int(sample_rate[:-1])

# Parameters
cost_w: float = 3 # €/m³, cost of water
cost_e: float = 0.05 # €/kWh, cost of electricity

# initial_datetime = '2024-01-08 10:55'
# final_datetime = '2024-01-08 14:00'

In [31]:
# Debug file
debug_file = src_path / "Eurecat/MODEL_ERRORS/dqn_drop_temperature.xlsx"

# Use debug_file parent folder as output path
output_path: Path = debug_file.parent / "attachments"
# Create folder attachments if it doesn't exist
output_path.mkdir(parents=True, exist_ok=True)

In [32]:
# Load configuration

with open( Path("data") / "variables_config.hjson") as f:
    vars_config = hjson.load(f)

# Load plots config
with open( Path("data") / "plot_config.hjson") as f:
    plt_config = hjson.load(f)

with open( Path("data") / "plot_config_validation.hjson") as f:
    plt_config_val = hjson.load(f)


In [33]:
# Read states

from solarMED_modeling import calculate_benefits

# Index column is the first one, drop it
df = pd.read_excel(debug_file, index_col=0) 

# Get sample rate
sample_rate_numeric = df['sample_time'][0]

# Create new columns that start with qmed*, qsf, qts*, qhx*, q3wv* to q*
# This should've been handled when exporting the data, model.to_dataframe(rename_flows=True) or if not, now it will export both options
for col in df.columns:
    # logger.debug(f"Processing column {col}")
    if re.match(r'mmed*', col) or re.match(r'msf*', col) or re.match(r'mhx*', col) or re.match(r'm3wv*', col) or re.match(r'mts*', col):
        # logger.debug(f"Column {col} matches, naming: q{col[1:]}")
        df[f'q{col[1:]}'] = df[col]

# Just once, rename Pts_out to Pts_dis, it's already been fixed in the model
# df.rename(columns={'Pts_out': 'Pts_dis'}, inplace=True)

# Just once, add Tts_h_out, it's already been fixed in the model
# df['Tts_h_out'] = df['Tts_h_t']
# df["Tts_c_in"] = df["Tmed_s_out"]

df = calculate_benefits(df, cost_w=cost_w, cost_e=cost_e, sample_rate_numeric=sample_rate_numeric)

In [34]:
# Visualize provided states

fig = experimental_results_plot(plt_config, df, vars_config=vars_config, resample=resample_figures)

fig.show(
    config=generate_plotly_config(fig, figure_name=f'solarMED_debugging_{debug_file.stem}')
)

In [35]:
# Save figure

save_figure(fig, f'solarMED_debugging_provided_data_{debug_file.stem}', output_path, formats=['html'])

In [36]:
# Try to reproduce given results

from solarMED_modeling.solar_med import SolarMED

%autoreload 2

span = 15
idx_start = span
df_mod = pd.DataFrame()

# Initialize past values, this should not be needed!!
# Prepend span nan values to the start of the dataframe
df = pd.concat([pd.DataFrame(np.nan, index=range(span), columns=df.columns), df], ignore_index=True)
# Fill the values for: Tsf_in, msf
df.loc[0:span-1, "Tsf_in"] = 36.09666667
df.loc[0:span-1, "qsf"] = 4.7


idx_end = len(df)

In [37]:
# Reproduce

# Initialize model
model = SolarMED(
    sample_time=sample_rate_numeric,
    resolution_mode='simple',
    
    # Initial states
    ## Thermal storage
    Tts_h=[df['Tts_h_t'].iloc[idx_start], df['Tts_h_m'].iloc[idx_start], df['Tts_h_b'].iloc[idx_start]], 
    Tts_c=[df['Tts_c_t'].iloc[idx_start], df['Tts_c_m'].iloc[idx_start], df['Tts_c_b'].iloc[idx_start]],
    
    ## Solar field
    Tsf_in_ant=df['Tsf_in'].iloc[idx_start-span:idx_start].values,
    msf_ant=df['qsf'].iloc[idx_start-span:idx_start].values,
    
    cost_w = 3, # €/m³ 
    cost_e = 0.05 # €/kWhe
)

# Run model
# %autoreload 2

for idx in range(idx_start, idx_end):
    # idx = 1
    ds = df.iloc[idx]
    
    logger.info(f"Iteration {idx} / {idx_end}")
    start_time = time.time()
        
    model.step(
        # Decision variables
        ## MED
        mmed_s=ds['qmed_s'],
        mmed_f=ds['qmed_f'],
        Tmed_s_in=ds['Tmed_s_in'],
        Tmed_c_out=ds['Tmed_c_out'],
        ## Thermal storage
        mts_src=ds['qhx_s'],
        ## Solar field
        Tsf_out=ds['Tsf_out'],
        # Tsf_out = 100,
        
        # Inputs
        # When the solar field is starting up, a flow can be provided to sync the model with the real system, if a valid Tsf_out is provided, it will be prioritized
        msf=ds['qsf'] if ds['qsf'] > 4 else None,
        
        # Environment variables
        Tmed_c_in=ds['Tmed_c_in'],
        Tamb=ds['Tamb'],
        I=ds['I'],
    )
    
    logger.info(f"Finished Iteration {idx} / {idx_end}, elapsed time: {time.time()-start_time:.2f} seconds. Current operation state is {model.operating_state.name}")

    df_mod = model.to_dataframe(df_mod, rename_flows=True)
    

In [38]:
# Visualize both together to see if they match

# Sync model index with measured data
df_mod.index = df.index[idx_start:idx if idx < idx_end - 1 else idx_end]

fig = experimental_results_plot(plt_config_val, df, df_comp=df_mod, vars_config=vars_config, resample=resample_figures)

fig.show(
    config=generate_plotly_config(fig, figure_name=f'solarMED_debugging_val_{debug_file.stem}')
)

In [40]:
# Save figure
save_figure(fig, f'solarMED_debugging_evaluation_result_{debug_file.stem}', output_path, formats=['html'])

In [4]:
# from simple_pid import PID
# import copy
# 
# pid = PID(1, 0.1, 0.05, setpoint=36.09666667)
# 
# pid_copy = copy.deepcopy(pid)
# 
# control_signal = pid_copy(100)
# 
# print(f'Are they independent? {pid is pid_copy}, pid.integral:{pid._integral} vs pid_copy.integral{pid_copy._integral}')